In [ ]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd

In [ ]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
#just for convinience
#Get data
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

train.head()    #Species are already defined numerically

train_y = train.pop('Species')  #remove label from the data
test_y = test.pop('Species')
train.head()
train.shape
train.keys()    #Gives us all the columns in the dataframe

In [ ]:
#We dont have epochs here. see more types of input functions to get an idea how they work and how to code
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)


In [ ]:
#feature columns
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))


In [ ]:
# Build a Deep Neural network (DNN) with 2 hidden layers with 30 and 10 hidden nodes each.
#tf.estimator stores a some premade models from tensor flow
classifier = tf.estimator.DNNClassifier(        
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)


#Training the model
#lambda is a single line function. It does whatever is written after :
#Eg:
# x = lambda: print("hi")
# x()
#Just like calling a function inside a function

classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),   #Similar to the last "function inside a function" in the previous code
    steps=5000) #Steps is similar to epochs


In [ ]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


In [ ]:
#Prediction for a single input

def input_fn(features, batch_size=256):
    #Convert input to a dataset without labels
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']

predict = {}

print("Please type numeric values as prompted")

for feature in features:
    valid = True
    while valid:
        val = input(feature + ": ")
        if not val.isdigit(): valid = False

    predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))

for pred_dict in predictions:
    print(pred_dict)
    class_id = pred_dict['class_ids'][0]
    print("Species is ", 3.SPECIES[class_id])
    probability = pred_dict['probabilities'][class_id]
    print("Probability is ", 100*probability)



Please type numeric values as prompted
SepalLength: 1.2
SepalWidth: .2
PetalLength: 5.2
PetalWidth: 3.1
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpd2l9yvdt/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([-0.9990073,  1.9139899,  3.7607677], dtype=float32), 'probabilities': array([0.00734583, 0.13525078, 0.85740334], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Species is  Virginica
Probability is  85.74033379554749
